In [3]:
# Imports
import numpy as np
import pandas as pd
from typing import List
from collections import OrderedDict

import torch
from torch.jit import RecursiveScriptModule


# other libraries
from typing import Final

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import os
print(os.getcwd())


c:\Users\anton\OneDrive\Escritorio\TERCERO\NLP\ProyectoFinal\rnn_explainability\src\AgnosticModels


In [10]:

# own modules
from src.model_utils import load_model
from src.model_utils import predict_single_text
from src.model_utils import predict_multiple_text
from src.model_utils import load_w2v_model

# permutation importance
from sklearn.inspection import permutation_importance

['c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\src\\AgnosticModels', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\.venv', '', 'c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\.venv\\Lib\\site-packages', 'c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\anton\\OneDrive\\Escritorio\\TERCERO\\NLP\\ProyectoFinal\\rnn_explainability\\.venv\\Lib\\site-packages\\Pythonwin', '/c:/Users/anton/OneDrive/Escritorio/TERCERO/NLP/ProyectoFinal/rnn_explainability/src', '/c:/Users/anton/OneDrive/Escritorio/TERCERO/NLP/ProyectoFinal/rnn_explainability/src', '/c:/Users/an

ImportError: attempted relative import with no known parent package

In [ ]:
# static variables
DATA_PATH: Final[str] = "NLP_Data/data"
MODEL_TYPE: Final[str] = "IMDB"  # "TweepFake"

# set device
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)

In [ ]:
# load the model
if MODEL_TYPE == "IMDB":
    model: RecursiveScriptModule = load_model("IMDB_best_model")
else:
    model: RecursiveScriptModule = load_model("best_model")
    
# Load the w2v model
w2vec_model = load_w2v_model()

In [ ]:
if MODEL_TYPE == "IMDB":
    file_path = DATA_PATH + '/test.txt'
    data: pd.DataFrame = pd.read_csv(file_path, sep='\t', header=None)
    data.columns = ['text', 'tag']

else:
    file_path = DATA_PATH + '/test.csv'
    data: pd.DataFrame = pd.read_csv(file_path)

    # replace the target column with a binary representation
    data['tag'] = data['account.type'].replace('human', 0)
    data['tag'] = data['tag'].replace('bot', 1)
    # Only keep columns text and tag
    data = data[['text', 'tag']]

print(data.head())

In [ ]:
# Test the model with a single text
text = data['text'][2]
print(text)
predicted = predict_single_text(text, model, device)
print(f"Predicted: {predicted}, Real: {data['tag'][0]}")

In [ ]:
def classifier_fn(text: str) -> int:
    predictions = predict_multiple_text(text, model, device, probability=True)
    # We have 2 classes. build the array (d, k) where d is the prediction and k is the number of classes
    array = []
    for prediction in predictions:
        array.append(np.array([1 - prediction, prediction]))
    array = np.array(array)
    return array

In [ ]:
class_names = [0, 1]
print(f"Explaining result for: {text}")
result = permutation_importance(classifier_fn, text, class_names, w2vec_model)
print(result)
